# Agenda

1. Pivot tables
2. Stack, unstack, melt
3. Joins
4. `filter` on columns

In [1]:
import numpy as np
import pandas as pd 
from pandas import Series, DataFrame

In [2]:
df = DataFrame(np.random.randint(0, 1000, [4,5]),
              index=list('abcd'),
              columns=list('vwxyz'))
df

,v,w,x,y,z
a,906,704,539,445,618
b,692,477,543,649,594
c,853,732,548,193,643
d,501,406,614,962,466


In [3]:
df.loc['e'] = [1,2,3,4,5]
df

,v,w,x,y,z
a,906,704,539,445,618
b,692,477,543,649,594
c,853,732,548,193,643
d,501,406,614,962,466
e,1,2,3,4,5


In [ ]:
new_row = 